In [1]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("lab1")
         .getOrCreate())

In [4]:
spark

In [5]:
!hdfs dfs -ls /labs/laba01/ml-100k/u.data

-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data


load data

In [6]:
rdd = sc.textFile("/labs/laba01/ml-100k/u.data").map(lambda x: x.split("\t"))

In [10]:
rdd.take(1)

[['196', '242', '3', '881250949']]

In [11]:
rdd.count()

100000

In [16]:
rdd_filter = rdd.filter(lambda x: x[1] == '22')

In [17]:
rdd_filter.count()

297

In [18]:
rdd_filter.filter(lambda x: x[2] == '1').count()

5

In [19]:
rdd_filter.filter(lambda x: x[2] == '2').count()

14

In [20]:
rdd_filter.filter(lambda x: x[2] == '3').count()

46

In [21]:
rdd_filter.filter(lambda x: x[2] == '4').count()

98

In [22]:
rdd_filter.filter(lambda x: x[2] == '5').count()

134

In [24]:
#all movies

In [25]:
rdd.filter(lambda x: x[2] == '1').count()

6110

In [26]:
rdd.filter(lambda x: x[2] == '2').count()

11370

In [27]:
rdd.filter(lambda x: x[2] == '3').count()

27145

In [28]:
rdd.filter(lambda x: x[2] == '4').count()

34174

In [29]:
rdd.filter(lambda x: x[2] == '5').count()

21201

In [112]:
result = sorted(rdd.flatMap(lambda x: x[2]).countByValue().items())

In [113]:
result

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]

In [114]:
result_filtered = sorted(rdd_filter.flatMap(lambda x: x[2]).countByValue().items())

In [123]:
result_filtered

[('1', 5), ('2', 14), ('3', 46), ('4', 98), ('5', 134)]

In [126]:
list(dict((x, y) for x, y in result_filtered).values())

[5, 14, 46, 98, 134]

In [131]:
json_dict = {"hist_film": list(dict((x, y) for x, y in result_filtered).values()),
       "hist_all": list(dict((x, y) for x, y in result).values())}

In [132]:
json_dict

{'hist_film': [5, 14, 46, 98, 134],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [133]:
import json

In [134]:
with open("lab01.json", "w") as json_file:
    json.dump(json_dict, json_file)